In [1]:
#import modules
from google.cloud import bigquery
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import requests
from pandas.io.json import json_normalize 
import numpy as np
import matplotlib
from sqlalchemy.types import Integer, Text, String, DateTime
import pandas_gbq as pd_gbq
from IPython.display import display, HTML

In [2]:
#authenticate with gbq

from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_info(
{
  "type": "",
  "project_id": "",
  "private_key_id": "",
  "private_key": "",
  "client_email": "",
  "client_id": "",
  "auth_uri": "",
  "token_uri": "",
  "auth_provider_x509_cert_url": "",
  "client_x509_cert_url": ""
})

# Update the in-memory credentials cache (added in pandas-gbq 0.7.0).
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "sportsdataio"

# The credentials and project_id arguments can be omitted.
#df = pandas_gbq.read_gbq("SELECT quantity FROM `test.woowoo`")
#print(df.head())

In [3]:
#TeamSeasonStats

#params

season = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentSeason?key=insertyourkey").json())
table_name = 'nfl.TeamSeasonStats'
project_id = 'sportsdataio'

#request

res = requests.get("https://api.sportsdata.io/v3/nfl/scores/json/TeamSeasonStats/" + season + "?key=insertyourkey").json()

#normalize json results to dataframe structure

TeamSeasonStats = json_normalize(res)

#load to gbq
pd_gbq.to_gbq(TeamSeasonStats, table_name, project_id=project_id, if_exists='replace')


In [ ]:
#allteams

#params

table_name = 'nfl.sdio_allteams'
project_id = 'sportsdataio'

#request

res = requests.get("https://api.sportsdata.io/v3/nfl/scores/json/AllTeams?key=insertyourkey").json()

#normalize json results to dataframe structure

allteams = json_normalize(res)
allteams.columns = allteams.columns.str.replace(".", "_")

#load to gbq
pd_gbq.to_gbq(allteams, table_name, project_id=project_id, if_exists='replace')

In [ ]:
#GameOddsByWeekOuter

#params

week = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek?key=insertyourkey").json())
season = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentSeason?key=insertyourkey").json())
nextweek = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/UpcomingWeek?key=insertyourkey").json())
table_name = 'nfl.sdio_gameoddsbyweekouter'
project_id = 'sportsdataio'

#request

res = requests.get("https://api.sportsdata.io/v3/nfl/odds/json/GameOddsByWeek/"+season+"/"+week+"?key=insertyourkey").json()

#normalize json results to dataframe structure

gameoddsbyweekouter = json_normalize(res)

gameoddsbyweekouter = gameoddsbyweekouter.drop(columns=['PregameOdds'])

#load to gbq
pd_gbq.to_gbq(gameoddsbyweekouter, table_name, project_id=project_id, if_exists='replace')

In [ ]:
#GameOddsByWeekInner

#params

week = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek?key=insertyourkey").json())
season = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentSeason?key=insertyourkey").json())
nextweek = str(requests.get("https://api.sportsdata.io/v3/nfl/scores/json/UpcomingWeek?key=insertyourkey").json())
table_name = 'nfl.sdio_gameoddsbyweekinner'
project_id = 'sportsdataio'

#request

res = requests.get("https://api.sportsdata.io/v3/nfl/odds/json/GameOddsByWeek/"+season+"/"+week+"?key=c2d1cb33627e4d769ba05cbb94dcec0c").json()

#normalize json results to dataframe structure

gameoddsbyweekinner = json_normalize(res,'PregameOdds')

#load to gbq
pd_gbq.to_gbq(gameoddsbyweekinner, table_name, project_id=project_id, if_exists='replace')

In [ ]:
#SeasonScores

#params

table_name = 'nfl.sdio_seasonscores'
project_id = 'sportsdataio'

#request

res = requests.get("https://api.sportsdata.io/v3/nfl/scores/json/Scores/2020REG?key=insertyourkey").json()

#normalize json results to dataframe structure

SeasonScores = json_normalize(res)
SeasonScores.columns = SeasonScores.columns.str.replace(".", "_")

#load to gbq
pd_gbq.to_gbq(SeasonScores, table_name, project_id=project_id, if_exists='replace')

In [90]:
#gameoddsbyweekinnertotal

#params
season = requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentSeason?key=insertyourkey").json()
week = requests.get("https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek?key=insertyourkey").json()
table_name = 'nfl.sdio_gameoddsbyweekinnertotal'
project_id = 'sportsdataio'

#create empty dataframe
raw_data = pd.DataFrame([])

#initialize loop
for i in range(1, week+1):
    r = requests.get("https://api.sportsdata.io/v3/nfl/odds/json/GameOddsByWeek/2020/"+str(i)+"?key=insertyourkey").json()
    data = json_normalize(r,'PregameOdds')
    df_temp = pd.DataFrame.from_dict(data)
    raw_data = raw_data.append(df_temp,ignore_index=True,sort=False)

#dataframe to gbq
pd_gbq.to_gbq(raw_data, table_name, project_id=project_id, if_exists='replace')

In [94]:
#ats calc

#params
project_id = 'sportsdataio'
table_name = 'nfl.sdio_ats'

#query
sql_expr = """

-- select historical consensus spreads for every week of nfl season 

with a as 
(SELECT 
a.ScoreId, 
a.HomePointSpread, 
a.AwayPointSpread,
a.OverUnder
FROM nfl.sdio_gameoddsbyweekinnertotal as a
where a.Sportsbook = 'Consensus'),

-- select historical season scores for every week of nfl season 
b as 
(
select * from nfl.sdio_seasonscores
),

-- merge weekly point spreads and weekly scores for comparison 

c as (
select 
a.*,
b.HomeScore,
b.AwayScore,
b.HomeScore+b.AwayScore as TotalScore,
b.HomeTeam, 
b.AwayTeam
from a
left join b on a.ScoreId = b.ScoreID
),

-- create actual point spread for home/away teams

d as (
select
c.*, 
c.AwayScore - c.HomeScore as home_spread_act,
c.HomeScore - c.AwayScore as away_spread_act

from c
),

-- create field that evaluates if the home/away team covered against the spread (ats)

e as (
select
d.*, 
case when d.home_spread_act < d.HomePointSpread then 1 else 0 end as home_cover,
case when d.away_spread_act < d.AwayPointSpread then 1 else 0 end as away_cover
from d ),

-- aggregate home_ats by team 

e1 as (
select
e.HomeTeam,
sum(e.home_cover) as sum
from e 
group by 1),

-- aggregate away_ats by team 

e2 as (
select
e.AwayTeam,
sum(e.away_cover) as sum
from e 
group by 1),

-- select all games from the season 

f as (
select a.Team,a.Games from nfl.TeamSeasonStats as a
),

-- merge all games with aggregate ats results
g as (
select 
f.Team,
f.Games,
e1.sum as homecover,
e2.sum as awaycover
from f
left join e1 on f.Team = e1.HomeTeam
left join e2 on f.Team = e2.AwayTeam
),

-- create ats_win and ats_loss 
h as (
select 
g.*,
(g.homecover+g.awaycover) as ats_win,
g.Games-(g.homecover+g.awaycover) as ats_loss
from g 
)

select * from h
"""

sql_df = pandas_gbq.read_gbq(sql_expr, project_id=project_id)

pd_gbq.to_gbq(sql_df, table_name, project_id=project_id, if_exists='replace')

In [4]:
#params
project_id = 'sportsdataio'

sql_expr = """
with a as 
(
-- get this week's games with detailed stats
SELECT 
*
FROM nfl.sdio_gameoddsbyweekinner as a
where a.Sportsbook = 'Consensus'),

b as (
--join this week's games summary stats to detailed stats 
SELECT
b.DateTime,
b.HomeTeamName,
b.AwayTeamName,
a.AwayPointSpread,
a.HomePointSpread,
a.OverUnder,
CASE when a.HomePointSpread > 0 then b.AwayTeamName else b.HomeTeamName end as favorite,
CASE when a.HomePointSpread < 0 then b.AwayTeamName else b.HomeTeamName end as underdog,
CASE when a.HomePointSpread < 0 then a.HomePointSpread else a.AwayPointSpread end as spread
FROM nfl.sdio_gameoddsbyweekouter as b 
left join a on a.ScoreId = b.ScoreId
),
--create implied points metric 
c as (
select
b.*, 
(b.OverUnder/2) - b.spread as fav_imp,
(b.OverUnder/2) + b.spread as dog_imp
from b),
--rankdef
rankdef as (
select
t.Team,
t.OpponentRushingYards,
t.OpponentPassingYards,
RANK () OVER (ORDER BY t.OpponentRushingYards asc) as rush_def_Rank,
RANK () OVER (ORDER BY t.OpponentPassingYards asc) as pass_def_Rank
from nfl.TeamSeasonStats as t
),
--ppg
ppg as (
select
a.Team,
a.Score,
a.Games,
(a.Score/a.Games) as ppg
from nfl.TeamSeasonStats as a
),
--ats
ats as 
(
select
a.Team,
cast(a.ats_win as float64)/cast(a.Games as float64) as cover_p
from nfl.sdio_ats as a
)


select 
c.*,
rda.rush_def_Rank as f_rush,
rda.pass_def_Rank as f_pass,
rdb.rush_def_Rank as d_rush,
rdb.pass_def_Rank as d_pass,
ppga.ppg as f_ppg, 
ppgb.ppg as d_ppg,
ppga.ppg+ ppgb.ppg as ppg_total, 
c.OverUnder - (ppga.ppg+ ppgb.ppg) as ppg_v_ou,
atsa.cover_p as f_cover_p, 
atsb.cover_p as d_cover_p
from c
left join rankdef as rda on c.favorite = rda.Team
left join rankdef as rdb on c.underdog = rdb.Team
left join ppg as ppga on c.favorite = ppga.Team
left join ppg as ppgb on c.underdog = ppgb.Team
left join ats as atsa on c.favorite = atsa.Team
left join ats as atsb on c.underdog = atsb.Team
order by 1 asc
"""

sql_df = pandas_gbq.read_gbq(sql_expr, project_id=project_id)

display(HTML(sql_df.to_html()))

,DateTime,HomeTeamName,AwayTeamName,AwayPointSpread,HomePointSpread,OverUnder,favorite,underdog,spread,fav_imp,dog_imp,f_rush,f_pass,d_rush,d_pass,f_ppg,d_ppg,ppg_total,ppg_v_ou,f_cover_p,d_cover_p
0,2020-12-10T20:20:00,LAR,NE,5.0,-5.0,43.5,LAR,NE,-5.0,26.75,16.75,7,3,28,15,25.000000,21.307692,46.307692,-2.807692,0.615385,0.461538
1,2020-12-13T13:00:00,CHI,HOU,-2.0,2.0,46.0,HOU,CHI,-2.0,25.00,21.00,31,21,17,17,24.000000,20.500000,44.500000,1.500000,0.333333,0.416667
2,2020-12-13T13:00:00,CIN,DAL,-3.0,3.0,43.0,DAL,CIN,-3.0,24.50,18.50,32,6,29,25,22.333333,19.750000,42.083333,0.916667,0.166667,0.583333
3,2020-12-13T13:00:00,MIA,KC,-7.0,7.0,50.5,KC,MIA,-7.0,32.25,18.25,26,12,20,14,30.833333,25.250000,56.083333,-5.583333,0.500000,0.750000
4,2020-12-13T13:00:00,NYG,ARI,-2.5,2.5,46.5,ARI,NYG,-2.5,25.75,20.75,21,16,3,18,27.666667,19.250000,46.916667,-0.416667,0.416667,0.666667
5,2020-12-13T13:00:00,TB,MIN,6.5,-6.5,53.0,TB,MIN,-6.5,33.00,20.00,1,22,18,26,28.666667,26.583333,55.250000,-2.250000,0.500000,0.500000
6,2020-12-13T13:00:00,CAR,DEN,3.5,-3.5,44.5,CAR,DEN,-3.5,25.75,18.75,15,19,25,9,23.333333,18.750000,42.083333,2.416667,0.583333,0.583333
7,2020-12-13T13:00:00,JAX,TEN,-7.5,7.5,52.5,TEN,JAX,-7.5,33.75,18.75,16,28,30,29,29.916667,20.916667,50.833333,1.666667,0.416667,0.500000
8,2020-12-13T16:05:00,LV,IND,-3.0,3.0,52.0,IND,LV,-3.0,29.00,23.00,6,8,19,24,27.333333,26.916667,54.250000,-2.250000,0.583333,0.583333
9,2020-12-13T16:05:00,SEA,NYJ,14.5,-14.5,47.5,SEA,NYJ,-14.5,38.25,9.25,4,32,11,31,29.416667,15.000000,44.416667,3.083333,0.500000,0.333333
